接下来就遇到了一个非常头痛的问题：不同序列的长度是有很大差别的。

比较早的品种，长度有20年以上，但短的品种可能只有一两年。

如果要追求覆盖面的广度，那么时间序列的长度就会缩短；

如果想要延长序列长度，那么能涵盖的品种范围就要缩小。

In [6]:
assets_list = [
    # 股指期货
    'IH.CFX', 'IF.CFX', 'IC.CFX',
    # 国债期货
    'TS.CFX', 'TF.CFX', 'T.CFX', 'TL1.CFX',
    # 黑色金属产业链
    'I.DCE', 'JM.DCE', 'RB.SHF', 'HC.SHF', 'SS.SHF', 'SF.ZCE', 'SM.ZCE',
    # 有色金属
    'CU.SHF', 'AL.SHF', 'ZN.SHF', 'NI.SHF',
    # 贵金属
    'AU.SHF', 'AG.SHF',
    # 能源化工
    'FU.SHF', 'LU.INE', 'BU.SHF', 'PG.DCE', 'TA.ZCE', 'EG.DCE', 'PF.ZCE', 
    'L.DCE', 'PP.DCE', 'V.DCE', 'EB.DCE', 'MA.ZCE', 'UR.ZCE', 'RU.SHF',
    # 农产品
    'A.DCE', 'B.DCE', 'M.DCE', 'RM.ZCE', 'Y.DCE', 'OI.ZCE', 'P.DCE', 'PK.ZCE',
    'C.DCE', 'CS.DCE', 'CF.ZCE', 'SR.ZCE', 'CJ.ZCE', 'AP.ZCE', 'SP.SHF', 
    'JD.DCE', 'LH.DCE',
    # 建材
    'FG.ZCE', 'SA.ZCE'
]


例如，上述品种中，尽管已经做了筛选，但最短的品种也只有4年左右的历史可用，再加上测试集还需要排除一部分，整体的数据量非常受限。

In [7]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
import os
os.chdir('d:/future/Index_Future_Prediction')

能不能在这个问题上做到二者的兼顾呢？

对此需要设计一套能处理变长输入的结构，分段来处理这个问题，首先选择最长的时间和最窄的范围，构造一个dataloader，然后再缩短长度，扩大范围，再构建新的dataloader。不断重复这个过程，就能得到一系列dataloader，数据的长度和宽度都不一样。

将这些dataloader合并，就可以做到对长短历史的兼顾。此外，这么做还有一个优势，就是对于长序列，其近期的历史被重复了多次，提高了权重，这对于解决模式迁移是有帮助的。

In [8]:
# 提取数据
seq_len = 120
batch_size = 64

feature_columns = ['inday_chg_open','inday_chg_high','inday_chg_low','inday_chg_close','inday_chg_amplitude', 'ma_10','ma_26','ma_45','ma_90','ma_vol',]
label_columns = ['label_return','down_prob','middle_prob','up_prob']

In [9]:
for i in range(13):

    start_date = 20100901 + i*10000
    end_date = 20230901

    feature = []
    label = []
    current_assets = []

    data = pd.read_csv(f'data/C.DCE.csv')
    data = data[data['trade_date'] > start_date].copy()
    data = data[data['trade_date'] < end_date].copy() # 20230901年以后数据不参与训练
    full_length = len(data)

    for asset_code in assets_list:
        data = pd.read_csv(f'data/{asset_code}.csv')
        data = data[data['trade_date'] > start_date]
        data = data[data['trade_date'] < end_date] # 所有2024年以后数据不参与训练
        # print(asset_code, len(data))
        if len(data) == full_length:
            current_assets.append(asset_code)
    print(start_date, len(current_assets))

20100901 17
20110901 18
20120901 18
20130901 20
20140901 23
20150901 29
20160901 36
20170901 37
20180901 38
20190901 41
20200901 45
20210901 50
20220901 53


因此，可以将数据分为3组，从2011年开始涵盖18个品种，从2016年开始涵盖36个品种，从2021年开始涵盖50个品种。

这里直接保存张量，供训练使用

In [10]:
start_date = 20210901
end_date = 20230901

feature = []
label = []
current_assets = []

data = pd.read_csv(f'data/C.DCE.csv')
data = data[data['trade_date'] > start_date].copy()
data = data[data['trade_date'] < end_date].copy()
full_length = len(data)

for asset_code in assets_list:
    data = pd.read_csv(f'data/{asset_code}.csv')
    data = data[data['trade_date'] > start_date]
    data = data[data['trade_date'] < end_date]
    if len(data) == full_length:
        current_assets.append(asset_code)
        feature.append(torch.tensor(data[feature_columns].values, dtype = torch.float32, device = 'cuda:0'))
        label.append(torch.tensor(data[label_columns].values, dtype = torch.float32, device = 'cuda:0'))

feature = torch.stack(feature, dim = 1)
label = torch.stack(label, dim = 1)
feature = feature.unfold(dimension = 0, size = seq_len, step = 1).transpose(2,3)
label = label[seq_len-1:]

# 记录当前可用的资产序号
assets_code = torch.tensor([assets_list.index(item) if item in assets_list else -1 for item in current_assets])

# 转化为weights并广播道跟feature相同维度
weights = torch.nn.functional.one_hot(assets_code, num_classes=53)
weights = weights.to(torch.float32)
weights = weights.broadcast_to(size = (feature.shape[0], *weights.shape))

print(feature.shape)
print(label.shape)
print(weights.shape)

torch.save(feature, f'data/feature_{start_date}')
torch.save(label, f'data/label_{start_date}')
torch.save(weights, f'data/weights_{start_date}')

torch.Size([365, 50, 120, 10])
torch.Size([365, 50, 4])
torch.Size([365, 50, 53])
